<a href="https://colab.research.google.com/github/JotaBlanco/TheValley/blob/main/EDA/02-analisis-exploratorio-visual/data_prep_previo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modificaciones sobre dataframes origen
Modificaciones a dataframes originales para prepararlos para EDA Visual.




# 01 Import

In [ ]:
import pandas as pd
import numpy as np

# 02 Load Data

In [ ]:
# df_casos

# url de archivo raw de casos confirmados
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'

# carga el archivo en un dataframe df_covid
df_casos = pd.read_csv(url)

df_casos.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,48718,48952,49161,49378,49621,49681,49817,50013,50190,50433,50655,50810,50886,51039,51280,51350,51405,51526,51526,51526,51526,53011,53105,53105,53207,53332,53400,53489,53538,53584,53584,53775,53831,53938,53984,54062,54141,54278,54403,54483
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,49191,50000,50637,51424,52004,52542,53003,53425,53814,54317,54827,55380,55755,56254,56572,57146,57727,58316,58316,58991,59438,59623,60283,61008,61705,62378,63033,63595,63971,64627,65334,65994,66635,67216,67690,67982,68568,69238,69916,70655
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,92597,93065,93507,93933,94371,94781,95203,95659,96069,96549,97007,97441,97857,98249,98631,98988,99311,99610,99897,100159,100408,100645,100873,101120,101382,101657,101913,102144,102369,102641,102860,103127,103381,103611,103833,104092,104341,104606,104852,105124
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7382,7382,7446,7466,7519,7560,7577,7602,7633,7669,7699,7756,7806,7821,7875,7919,7983,8049,8117,8166,8192,8249,8308,8348,8348,8489,8586,8586,8586,8682,8818,8868,8946,9038,9083,9083,9194,9308,9379,9416
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16277,16362,16407,16484,16562,16626,16644,16686,16802,16931,17029,17099,17149,17240,17296,17371,17433,17553,17568,17608,17642,17684,17756,17864,17974,18066,18156,18193,18254,18343,18425,18613,18679,18765,18875,18926,19011,19093,19177,19269


In [ ]:
# df_muertes

# url de archivo muertes 
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'

# carga el archivo en un dataframe df_covid
df_muertes = pd.read_csv(url)

df_muertes.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,...,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1969,1995,2011,2025,2030,2047,2067,2082,2096,2117,2126,2139,2149,2160,2174,2179,2181,2191,2191,2191,2191,2237,2244,2244,2253,2257,2264,2277,2288,2301,2301,2314,2324,2336,2339,2343,2346,2354,2363,2370
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1016,1028,1040,1055,1066,1074,1088,1098,1111,1117,1125,1134,1143,1153,1164,1170,1174,1181,1181,1190,1193,1199,1210,1217,1223,1230,1233,1241,1247,1252,1256,1261,1265,1270,1277,1281,1287,1291,1296,1303
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2609,2623,2631,2640,2647,2659,2666,2675,2687,2696,2705,2716,2722,2728,2737,2745,2751,2756,2762,2769,2772,2777,2782,2786,2792,2798,2803,2807,2812,2816,2819,2822,2827,2831,2836,2840,2843,2849,2853,2856
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,79,79,79,79,80,80,81,81,82,82,83,83,83,83,83,84,84,84,84,84,84,84,84,84,84,84,85,85,85,86,87,88,88,91,91,91,92,92,93,93
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,372,372,379,382,384,386,387,390,393,393,393,396,399,399,403,403,405,405,405,407,408,408,410,413,413,415,416,416,420,422,424,425,428,431,436,439,442,444,448,452


# 03 Transformaciones dataset Covid-19
Queremos hacer un análisis visual que incluya a España, USA, Francia, Reino Unido, Francia, Italia, Portugal y Alemania. Así que como paso previo, tendremos que crear un nuevo dataframe df_covid_clean sin el resto de países.

Nota: Algunos de los países deseados tienen sus datos desagregados en varias provincias. Agrúpalos en una sola entrada. Es decir, no queremos en este análisis tener una fila que representa a Nueva York y otra que representa a California, sino una única que representa a USA donde se incluyen ambas. De hecho, puedes eliminar la columna Provincia del dataframe df_covid_clean.

In [ ]:
# Países totales
df_casos['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia',
       'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark',
       'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic',
       'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea',
       'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France',
       'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece',
       'Grenada', 'Guatemala', 'Guin

In [ ]:
# Lista con los paises deseados
lista_paises = ['China', 
                'France', 'Germany', 'Italy', 
                'Portugal', 'Spain', 'US', 'United Kingdom']

In [ ]:
def extraer_paises(df, lista_paises):
  
  # Paises que contienen una fila con su total de casos 
  # (independientemente de que tenga otras filas dedicadas a regiones específicas)
  paises_listos = ((df['Country/Region'].isin(lista_paises))&
                   (df['Province/State'].isna()))
  
  # Nos quedamos con esas filas
  df_1 = df.loc[paises_listos, :]

  # Cuáles son estos paises?
  lista_paises_listos = sorted(df_1['Country/Region'].unique())

  # Finalizamos el formato
  df_1_final = df_1.iloc[:,4:].T
  df_1_final.columns = ['UK' if pais=='United Kingdom' else pais for pais in lista_paises_listos]
  df_1_final.index = pd.to_datetime(df_1.columns[4:],format='%m/%d/%y')

  # Resto de países: 
  # aquellos que tienen varias regiones pero no una fila con el total para el país
  lista_resto_paises = sorted(set(lista_paises) - set(lista_paises_listos))

  # Generamos una fila con la información acumulada de todo el país
  df_2 = df.loc[df['Country/Region'].isin(lista_resto_paises)]
  cols_fechas = df_2.columns[4:]
  df_2 = df_2.groupby('Country/Region')[cols_fechas].sum().reset_index()

  # Finalizamos el formato
  df_2_final = df_2.iloc[:,1:].T
  df_2_final.columns = sorted(lista_resto_paises)

  # Dataframe final
  df_final = pd.concat([df_1_final, df_2_final], axis=1)
  return df_final[sorted(df_final.columns)]

In [ ]:
df_casos_limpio = extraer_paises(df_casos, lista_paises)
df_casos_limpio.head()

,China,France,Germany,Italy,Portugal,Spain,UK,US
2020-01-22,548,0,0,0,0,0,0,1
2020-01-23,643,0,0,0,0,0,0,1
2020-01-24,920,2,0,0,0,0,0,2
2020-01-25,1406,3,0,0,0,0,0,2
2020-01-26,2075,3,0,0,0,0,0,5


In [ ]:
df_muertes_limpio = extraer_paises(df_muertes, lista_paises)
df_muertes_limpio.head()

,China,France,Germany,Italy,Portugal,Spain,UK,US
2020-01-22,17,0,0,0,0,0,0,0
2020-01-23,18,0,0,0,0,0,0,0
2020-01-24,26,0,0,0,0,0,0,0
2020-01-25,42,0,0,0,0,0,0,0
2020-01-26,56,0,0,0,0,0,0,0


# 04 Añadimos variables

## 0401 Relativización de cifras
A tasas de casos/fallecimientos por cada 100.000 habitantes.

In [ ]:
# Importamos el archivos usando read_csv()
dir_archivo = 'https://raw.githubusercontent.com/datasets/population/master/data/population.csv'
df_pob = pd.read_csv(dir_archivo)
df_pob = df_pob[df_pob['Country Name'].isin(lista_paises+['United States', 'USA'])]
df_pob = df_pob.groupby('Country Name')['Value'].last().reset_index()
df_pob.loc[df_pob['Country Name']=='United Kingdom', 'Country Name'] = 'UK'
df_pob.loc[df_pob['Country Name']=='United States', 'Country Name'] = 'US'
df_pob

,Country Name,Value
0,China,1392730000
1,France,66977107
2,Germany,82905782
3,Italy,60421760
4,Portugal,10283822
5,Spain,46796540
6,UK,66460344
7,US,326687501


In [ ]:
for country in df_pob['Country Name'].values:
  print(country)
  print(int(df_pob.loc[df_pob['Country Name']==country, 'Value'].values))

China
1392730000
France
66977107
Germany
82905782
Italy
60421760
Portugal
10283822
Spain
46796540
UK
66460344
US
326687501


In [ ]:
def añadir_cols_rel(df, df_pob):
  for country in [col for col in df.columns if '_REL' not in col]:
    pob_country = int(df_pob.loc[df_pob['Country Name']==country, 'Value'].values)
    df[country+'_REL'] = np.round(df[country]/pob_country*100000, 2)
  return df

In [ ]:
df_casos_limpio = añadir_cols_rel(df_casos_limpio, df_pob)
df_muertes_limpio = añadir_cols_rel(df_muertes_limpio, df_pob)

In [ ]:
display(df_casos_limpio.head())
display(df_muertes_limpio.head())

,China,France,Germany,Italy,Portugal,Spain,UK,US,China_REL,France_REL,Germany_REL,Italy_REL,Portugal_REL,Spain_REL,UK_REL,US_REL
2020-01-22,548,0,0,0,0,0,0,1,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-23,643,0,0,0,0,0,0,1,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,920,2,0,0,0,0,0,2,0.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-25,1406,3,0,0,0,0,0,2,0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-26,2075,3,0,0,0,0,0,5,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,China,France,Germany,Italy,Portugal,Spain,UK,US,China_REL,France_REL,Germany_REL,Italy_REL,Portugal_REL,Spain_REL,UK_REL,US_REL
2020-01-22,17,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-23,18,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-24,26,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-25,42,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2020-01-26,56,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 0402 Últimos 7 días
Cifras acumuladas en últimos 7 días.

In [ ]:
# Nuevos casos ultimos 7 días
for col in df_casos_limpio.columns:
  df_casos_limpio[col+'_ult7'] = df_casos_limpio[col] - df_casos_limpio[col].shift(7)

# Nuevas muertes ultimos 7 días
for col in df_muertes_limpio.columns:
  df_muertes_limpio[col+'_ult7'] = df_muertes_limpio[col] - df_muertes_limpio[col].shift(7)

In [ ]:
# Añadimos col fecha
df_casos_limpio.insert(0, 'Date', df_casos_limpio.index)
df_casos_limpio.reset_index(drop=True, inplace=True)
df_casos_limpio.head()

,Date,China,France,Germany,Italy,Portugal,Spain,UK,US,China_REL,France_REL,Germany_REL,Italy_REL,Portugal_REL,Spain_REL,UK_REL,US_REL,China_ult7,France_ult7,Germany_ult7,Italy_ult7,Portugal_ult7,Spain_ult7,UK_ult7,US_ult7,China_REL_ult7,France_REL_ult7,Germany_REL_ult7,Italy_REL_ult7,Portugal_REL_ult7,Spain_REL_ult7,UK_REL_ult7,US_REL_ult7
0,2020-01-22,548,0,0,0,0,0,0,1,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,643,0,0,0,0,0,0,1,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,920,2,0,0,0,0,0,2,0.07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-25,1406,3,0,0,0,0,0,2,0.10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-26,2075,3,0,0,0,0,0,5,0.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Añadimos col fecha
df_muertes_limpio.insert(0, 'Date', df_muertes_limpio.index)
df_muertes_limpio.reset_index(drop=True, inplace=True)
df_muertes_limpio.head()

,Date,China,France,Germany,Italy,Portugal,Spain,UK,US,China_REL,France_REL,Germany_REL,Italy_REL,Portugal_REL,Spain_REL,UK_REL,US_REL,China_ult7,France_ult7,Germany_ult7,Italy_ult7,Portugal_ult7,Spain_ult7,UK_ult7,US_ult7,China_REL_ult7,France_REL_ult7,Germany_REL_ult7,Italy_REL_ult7,Portugal_REL_ult7,Spain_REL_ult7,UK_REL_ult7,US_REL_ult7
0,2020-01-22,17,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-23,18,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-24,26,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-25,42,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-26,56,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Primero emparejamos Colab con Google Drive
from google.colab import drive
drive.mount('/content/drive')
df_casos_limpio.to_csv(f"/content/drive/My Drive/Data/Clases/Covid_Casos_Clean.csv", sep=",", index=False)
df_muertes_limpio.to_csv(f"/content/drive/My Drive/Data/Clases/Covid_Muertes_Clean.csv", sep=",", index=False)

Mounted at /content/drive
